<a href="https://colab.research.google.com/github/Yash-Patil09/Machine-Learning-Algo/blob/main/Fake_News_Detection(NLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/content/Fake News Detection.csv')

In [6]:
df.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


In [7]:
df.shape

(4009, 4)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   URLs      4009 non-null   object
 1   Headline  4009 non-null   object
 2   Body      3988 non-null   object
 3   Label     4009 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 125.4+ KB


In [9]:
df.isnull().sum()

,0
URLs,0
Headline,0
Body,21
Label,0


In [11]:
df = df.dropna()
df.isnull().sum()

,0
URLs,0
Headline,0
Body,0
Label,0


In [12]:
df.shape

(3988, 4)

In [13]:
df['Label'].shape

(3988,)

In [14]:
df.loc[:,'text'] = df['Headline'].fillna('')+" "+df["Body"].fillna('')

In [15]:
df['text']

,text
0,Four ways Bob Corker skewered Donald Trump Ima...
1,Linklater's war veteran comedy speaks to moder...
2,Trump’s Fight With Corker Jeopardizes His Legi...
3,Egypt's Cheiron wins tie-up with Pemex for Mex...
4,Jason Aldean opens 'SNL' with Vegas tribute Co...
...,...
4003,CNN and Globalist Exposed - Steve Quayle and A...
4004,Trends to Watch Trends to Watch\n% of readers ...
4005,Trump Jr. Is Soon To Give A 30-Minute Speech F...
4007,China to accept overseas trial data in bid to ...


In [21]:
df.dropna()
print(df.isnull().sum())
print(f'Data shape: {df.shape}')

URLs        0
Headline    0
Body        0
Label       0
text        0
dtype: int64
Data shape: (3988, 5)


In [22]:
df = df.drop(columns=['Headline','Body','URLs'])
df.head()

,Label,text
0,1,Four ways Bob Corker skewered Donald Trump Ima...
1,1,Linklater's war veteran comedy speaks to moder...
2,1,Trump’s Fight With Corker Jeopardizes His Legi...
3,1,Egypt's Cheiron wins tie-up with Pemex for Mex...
4,1,Jason Aldean opens 'SNL' with Vegas tribute Co...


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('text').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Label']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Label'}, axis=1)
              .sort_values('Label', ascending=True))
  xs = counted['Label']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_1.sort_values('Label', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('text')):
  _plot_series(series, series_name, i)
  fig.legend(title='text', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Label')
_ = plt.ylabel('count()')

In [23]:
from sklearn.model_selection import train_test_split


In [24]:
X = df['text']
y = df['Label']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
print(f'X train shape:{X_train.shape}')
print(f'X test shape:{X_test.shape}')
print(f'y train shape:{y_train.shape}')
print(f'y test shape:{y_test.shape}')

X train shape:(3190,)
X test shape:(798,)
y train shape:(3190,)
y test shape:(798,)


In [26]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [31]:
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [32]:
# Initialize stopwords, stemmer, and lemmatizer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [33]:
def preprocess_text(text):
    words = word_tokenize(text.lower())
    words = [word for word in words if word.isalnum() and word not in stop_words]
    words = [stemmer.stem(word) for word in words]  # Stemming
    words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatization
    return " ".join(words)

In [34]:
X_train = X_train.apply(preprocess_text)
X_test = X_test.apply(preprocess_text)

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [36]:
vectorizer = TfidfVectorizer()
X_train_V = vectorizer.fit_transform(X_train)
X_test_V = vectorizer.transform(X_test)

Model Training

In [37]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [38]:
nb = MultinomialNB()
nb.fit(X_train_V,y_train)

MultinomialNB()

In [40]:
nb_prediction = nb.predict(X_test_V)

In [41]:
nb_accuracy = accuracy_score(y_test, nb_prediction)
print(f'Naïve Bayes Accuracy: {nb_accuracy:.4f}')

Naïve Bayes Accuracy: 0.9148


Random Forest

In [42]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model = rf_model.fit(X_train_V, y_train)
rf_prediction = rf_model.predict(X_test_V)
rf_accuracy = accuracy_score(y_test, rf_prediction)
print(f'Random Forest Accuracy: {rf_accuracy:.4f}')

Random Forest Accuracy: 0.9586


In [44]:
# Manual testing
def manual_test(text_input):
    # Preprocess the input text
    preprocessed_input = preprocess_text(text_input)

    # Vectorize the preprocessed text
    input_vectorized = vectorizer.transform([preprocessed_input])

    # Predict using Naive Bayes
    nb_prediction_manual = nb.predict(input_vectorized)

    # Predict using Random Forest
    rf_prediction_manual = rf_model.predict(input_vectorized)

    print(f"Input text: {text_input}")
    print(f"Preprocessed text: {preprocessed_input}")
    print(f"Naive Bayes Prediction: {'Fake' if nb_prediction_manual[0] == 0 else 'Real'}")
    print(f"Random Forest Prediction: {'Fake' if rf_prediction_manual[0] == 0 else 'Real'}")
    print("-" * 30) # Add a separator for clarity

# Example usage: Replace with the text you want to test
test_text_1 = "This is a test article about something that might be fake or real."
manual_test(test_text_1)

test_text_2 = "Breaking News: Scientists discover a new planet made of diamond."
manual_test(test_text_2)

test_text_3 = "President gives speech on the economy and job growth."
manual_test(test_text_3)

test_text_4 = "Exclusive: Secret documents reveal alien cover-up by the government."
manual_test(test_text_4)

Input text: This is a test article about something that might be fake or real.
Preprocessed text: test articl someth might fake real
Naive Bayes Prediction: Fake
Random Forest Prediction: Real
------------------------------
Input text: Breaking News: Scientists discover a new planet made of diamond.
Preprocessed text: break news scientist discov new planet made diamond
Naive Bayes Prediction: Fake
Random Forest Prediction: Fake
------------------------------
Input text: President gives speech on the economy and job growth.
Preprocessed text: presid give speech economi job growth
Naive Bayes Prediction: Real
Random Forest Prediction: Real
------------------------------
Input text: Exclusive: Secret documents reveal alien cover-up by the government.
Preprocessed text: exclus secret document reveal alien govern
Naive Bayes Prediction: Fake
Random Forest Prediction: Real
------------------------------
